In [5]:
import os
from dotenv import load_dotenv
import pandas as pd
import requests
import time
from itertools import islice
from tqdm import tqdm
import json
load_dotenv()

BASE_URL_AB = 'https://acousticbrainz.org'
BASE_URL_MB = 'https://musicbrainz.org/ws/2'
lastfm_tracks = pd.read_csv('data/lastfm/tracks_df.csv')
lastfm_tracks_mbids = lastfm_tracks['mbid'].unique()

# Rate limit of 1 request per second is advised by the website

In [119]:
lastfm_tracks[lastfm_tracks['mbid'] == '15eb9506-dc2a-427c-bbc3-9eac60e47991']

,name,mbid,url,duration,streamable,listeners,playcount,artist,album,toptags,wiki
21,Bestrafe Mich,15eb9506-dc2a-427c-bbc3-9eac60e47991,https://www.last.fm/music/Rammstein/_/Bestrafe...,219000,"{'#text': '0', 'fulltrack': '0'}",298297,2173676,"{'name': 'Rammstein', 'mbid': 'b2d122f9-eadb-4...","{'artist': 'Rammstein', 'title': 'Sehnsucht', ...","{'tag': [{'name': 'industrial metal', 'url': '...","{'published': '24 Jul 2008, 12:30', 'summary':..."


{'recording_ids': 'eddcdc3a-8e1b-4a5c-acf1-855e817627e8;aa82cd16-2777-4bd2-8068-851810fadae6;425c61b8-1d9c-4a87-a9b4-ab2a4262bac9;6662c57d-240c-471a-b360-fcba24b03142;fa1b4198-5e13-4c4c-9b7b-2b6594c2f679;50280310-022b-44bd-8561-92bc1a552969;76341a6e-bac9-4ab3-9d9a-3cf1c9ceac80;0cb01668-8ba7-42b1-b447-34e5f6e1b9c0;2d850c0b-3e21-4662-a6b6-5a15a1e677f4;d1d497aa-2c55-4341-b046-c9d6a6845dfc;0b29c009-6cf9-47c1-bf3d-935b54d8e4bc;079602c1-ac46-44c2-9bb2-841195e07c96;a901c950-4b8e-4abd-8efa-71def5f171b5;9b66012f-efa7-470a-a08f-22e24c537164;b32a447d-2ac1-400e-aece-e8a1bfe276b1;2c0b2787-307c-49e9-b2e9-5b1ddd6e713e;a89a7113-43d8-489a-9031-bb7e6fc49cf1;ae28e0d1-73ab-4292-a8f5-fa28dffec415;454d3f46-fa77-4d3a-93b7-0987ef19f88c;6bcd5cbb-d1fe-4023-b367-61535e8e5c6c;9619dcfa-7bb8-4956-927f-0931645bceaa;e465471c-4a41-47e5-a6f8-3d618c9607dc;78b41371-be0d-4325-87b3-9c67582e9f79;c16dcdcd-e24b-445d-b02b-89d9e0269a27;cae65223-805a-4f31-8d4a-eb39044dcff1'}


In [126]:
params = {'recording_ids': 'eddcdc3a-8e1b-4a5c-acf1-855e817627e8;aa82cd16-2777-4bd2-8068-851810fadae6;425c61b8-1d9c-4a87-a9b4-ab2a4262bac9;6662c57d-240c-471a-b360-fcba24b03142;fa1b4198-5e13-4c4c-9b7b-2b6594c2f679;50280310-022b-44bd-8561-92bc1a552969;76341a6e-bac9-4ab3-9d9a-3cf1c9ceac80;0cb01668-8ba7-42b1-b447-34e5f6e1b9c0;2d850c0b-3e21-4662-a6b6-5a15a1e677f4;d1d497aa-2c55-4341-b046-c9d6a6845dfc;0b29c009-6cf9-47c1-bf3d-935b54d8e4bc;079602c1-ac46-44c2-9bb2-841195e07c96;a901c950-4b8e-4abd-8efa-71def5f171b5;9b66012f-efa7-470a-a08f-22e24c537164;b32a447d-2ac1-400e-aece-e8a1bfe276b1;2c0b2787-307c-49e9-b2e9-5b1ddd6e713e;a89a7113-43d8-489a-9031-bb7e6fc49cf1;ae28e0d1-73ab-4292-a8f5-fa28dffec415;454d3f46-fa77-4d3a-93b7-0987ef19f88c;6bcd5cbb-d1fe-4023-b367-61535e8e5c6c;9619dcfa-7bb8-4956-927f-0931645bceaa;e465471c-4a41-47e5-a6f8-3d618c9607dc;78b41371-be0d-4325-87b3-9c67582e9f79;c16dcdcd-e24b-445d-b02b-89d9e0269a27;cae65223-805a-4f31-8d4a-eb39044dcff1'}


{'recording_ids': 'eddcdc3a-8e1b-4a5c-acf1-855e817627e8;aa82cd16-2777-4bd2-8068-851810fadae6;425c61b8-1d9c-4a87-a9b4-ab2a4262bac9;6662c57d-240c-471a-b360-fcba24b03142;fa1b4198-5e13-4c4c-9b7b-2b6594c2f679;50280310-022b-44bd-8561-92bc1a552969;76341a6e-bac9-4ab3-9d9a-3cf1c9ceac80;0cb01668-8ba7-42b1-b447-34e5f6e1b9c0;2d850c0b-3e21-4662-a6b6-5a15a1e677f4;d1d497aa-2c55-4341-b046-c9d6a6845dfc;0b29c009-6cf9-47c1-bf3d-935b54d8e4bc;079602c1-ac46-44c2-9bb2-841195e07c96;a901c950-4b8e-4abd-8efa-71def5f171b5;9b66012f-efa7-470a-a08f-22e24c537164;b32a447d-2ac1-400e-aece-e8a1bfe276b1;2c0b2787-307c-49e9-b2e9-5b1ddd6e713e;a89a7113-43d8-489a-9031-bb7e6fc49cf1;ae28e0d1-73ab-4292-a8f5-fa28dffec415;454d3f46-fa77-4d3a-93b7-0987ef19f88c;6bcd5cbb-d1fe-4023-b367-61535e8e5c6c;9619dcfa-7bb8-4956-927f-0931645bceaa;e465471c-4a41-47e5-a6f8-3d618c9607dc;78b41371-be0d-4325-87b3-9c67582e9f79;c16dcdcd-e24b-445d-b02b-89d9e0269a27;cae65223-805a-4f31-8d4a-eb39044dcff1'}

In [123]:
endpoint = f"{BASE_URL_AB}/api/v1/high-level"

ab_data = []
raw_ab_data = {}

batch_size = 25
for i in tqdm(range(0, len(lastfm_tracks_mbids), batch_size), desc="Processing batches"):
    batch = lastfm_tracks_mbids[i:i + batch_size]
    params = {
        'recording_ids': ';'.join(batch),
    }

    response_json = requests.get(endpoint, params=params).json()
    raw_ab_data.update(response_json)
    for mbid, data in response_json.items():
        if mbid not in lastfm_tracks_mbids: 
            continue
        features = {
            'mbid': mbid,
            'isrcs': data['0']['metadata'].get('tags', {}).get('isrc', []),
            'danceability': data['0']['highlevel'].get('danceability', {}).get('all', {}).get('danceable', None),
            'genre_alternative': data['0']['highlevel'].get('genre_dortmund', {}).get('all', {}).get('alternative',None),
            'genre_blues': data['0']['highlevel'].get('genre_dortmund', {}).get('all', {}).get('blues',None),
            'genre_electronic': data['0']['highlevel'].get('genre_dortmund', {}).get('all', {}).get('electronic',None),
            'genre_folkcountry': data['0']['highlevel'].get('genre_dortmund', {}).get('all', {}).get('folkcountry',None),
            'genre_funksoulrnb': data['0']['highlevel'].get('genre_dortmund', {}).get('all', {}).get('funksoulrnb',None),
            'genre_jazz': data['0']['highlevel'].get('genre_dortmund', {}).get('all', {}).get('jazz',None),
            'genre_pop': data['0']['highlevel'].get('genre_dortmund', {}).get('all', {}).get('pop',None),
            'genre_raphiphop': data['0']['highlevel'].get('genre_dortmund', {}).get('all', {}).get('raphiphop',None),
            'genre_rock': data['0']['highlevel'].get('genre_dortmund', {}).get('all', {}).get('rock',None),
            'mood_happy': data['0']['highlevel'].get('mood_happy', {}).get('all', {}).get('happy', None),
            'mood_party': data['0']['highlevel'].get('mood_party', {}).get('all', {}).get('party', None),
            'mood_relaxed': data['0']['highlevel'].get('mood_relaxed', {}).get('all', {}).get('relaxed', None),
            'mood_sad': data['0']['highlevel'].get('mood_sad', {}).get('all', {}).get('sad', None),
            'metadata': data['0']['metadata']['tags']
        }
        ab_data.append(features)
    time.sleep(0.2)

ab_df = pd.DataFrame(ab_data)



Processing batches: 100%|██████████| 181/181 [05:38<00:00,  1.87s/it]


In [127]:
ab_df

,mbid,isrcs,danceability,genre_alternative,genre_blues,genre_electronic,genre_folkcountry,genre_funksoulrnb,genre_jazz,genre_pop,genre_raphiphop,genre_rock,mood_happy,mood_party,mood_relaxed,mood_sad,metadata
0,15eb9506-dc2a-427c-bbc3-9eac60e47991,[],9.836463e-01,1.073386e-01,2.366554e-02,0.758375,0.023443,3.764465e-03,0.005285,4.931943e-03,2.200001e-03,7.099602e-02,0.406969,0.000018,0.808817,0.057036,"{'album': ['Sehnsucht'], 'albumartist': ['Ramm..."
1,2119f5c9-1954-4aed-9636-e3c2f65214c3,[USGR19906511],6.906841e-01,3.149457e-03,9.618337e-03,0.966064,0.011060,1.665185e-04,0.007583,2.062194e-04,9.586730e-05,2.056043e-03,0.191526,0.280979,0.733868,0.387445,{'acoustid_id': ['73a4c17a-d642-491b-a7c1-93d0...
2,2bfbc6ff-71c5-4af0-a08c-f005e2b725bc,[],3.000001e-14,1.988054e-08,1.783436e-08,0.999985,0.000002,2.021804e-07,0.000013,2.654895e-07,4.740200e-08,2.055116e-07,0.341914,0.000016,0.808817,0.161756,"{'album': ['Scott Pilgrim vs. The World'], 'al..."
3,2d6f51c6-afc5-457e-9c58-689679c08d30,"[USCA21001266, USCA21102081]",9.979379e-01,2.081701e-02,2.269153e-03,0.969157,0.004214,1.854199e-04,0.000548,8.294979e-04,1.509488e-04,1.829315e-03,0.828405,0.396032,0.163727,0.206887,{'acoustid_id': ['48e85eaa-580e-476e-bf80-cf80...
4,2df08900-e255-4a99-9f3b-ed10c338b078,[USGF19141504],2.746909e-01,2.608671e-02,2.856929e-03,0.964315,0.003467,1.513081e-04,0.000419,4.036218e-04,1.112447e-04,2.189646e-03,0.768490,0.800792,0.159348,0.267005,{'acoustid_id': ['e9b744cb-0893-4ce8-a556-c88f...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4402,dfac3d5d-a114-4bbb-a8fd-0ebf08ac5741,"[GB0028732800, GBAJH0600230]",6.284546e-01,5.552276e-03,5.708538e-04,0.992245,0.000856,1.602205e-05,0.000177,4.463885e-05,8.190442e-06,5.302191e-04,0.546749,0.818965,0.037377,0.253111,{'acoustid_id': ['44c4945d-ad83-4de5-b779-ed77...
4403,e53b9c87-fa13-4b9d-b727-85e1b8d01eea,[],9.963169e-01,7.011929e-02,3.816643e-02,0.677682,0.087407,7.021863e-03,0.044888,1.274968e-02,6.546235e-03,5.541874e-02,0.206582,0.568529,0.603327,0.338235,{'acoustid_fingerprint': ['AQADtHo0pdiLdEyS6NC...
4404,f1f3e674-e205-4b5f-8577-20590aba1072,[],9.706204e-01,1.346382e-01,2.371619e-02,0.747442,0.047635,2.947790e-03,0.004374,5.764775e-03,1.872289e-03,3.160936e-02,0.828395,0.746855,0.039698,0.137551,"{'album': ['Twenty One'], 'albumartist': ['Mys..."
4405,f33fc4cb-0b10-4666-9c12-20ec39ea71d7,[],9.540115e-02,2.834072e-02,4.183971e-03,0.951392,0.011040,2.379440e-04,0.000612,7.986635e-04,1.867631e-04,3.207383e-03,0.652600,0.337524,0.888201,0.279564,"{'album': ['Lazy Eye'], 'artist': ['Silversun ..."


In [77]:
# with open('data/brainz/ab_data_raw.json', 'w') as f:
#     json.dump(ab_data, f)

# with open('data/brainz/ab_data_raw.json', 'r') as f:
#     ab_data = json.load(f)


In [113]:
lookups = ';'.join(list(raw_ab_data.keys())[:10])

In [114]:
endpoint = f"{BASE_URL_MB}/recording/"

mb_data = []


batch_size = 25  
for i in tqdm(range(0, len(list(raw_ab_data.keys())), batch_size), desc="Processing MBIDs"):
    batch = list(raw_ab_data.keys())[i:i + batch_size]
    

    mbid_query = " OR ".join([f"mbid:{mbid}" for mbid in batch])
    
    params = {
        "query": mbid_query,
        "fmt": "json"
    }

    headers = {"User-Agent": "JHU-Project/1.0 (spalit2@jh.edu)"}

    response = requests.get(endpoint, params=params, headers=headers)

    mb_data.extend(response.json()['recordings'])

    time.sleep(0.2)


Processing MBIDs: 100%|██████████| 177/177 [03:24<00:00,  1.16s/it]


In [115]:
mb_df = pd.DataFrame(mb_data)
# mb_df.to_csv('data/brainz/mb_data.csv', index=False)

In [116]:
mb_df

,id,score,title,length,disambiguation,video,artist-credit,first-release-date,releases,tags,isrcs
0,a1f79f80-7190-4881-ad82-724a351da3d9,100,Golden Skans,197000.0,"live, The Black Sessions",None,"[{'name': 'Klaxons', 'artist': {'id': '63011a8...",2006-11-12,[{'id': '6c91736e-8179-41e8-8a31-99b4e28840f7'...,NaN,NaN
1,3245d20e-b10a-4316-8ef2-ebd1b918a25a,100,36 Degrees,183000.0,"live, 2000-08-06: FIB, Benicàssim, Spain",None,"[{'name': 'Placebo', 'artist': {'id': '847e828...",2000,[{'id': '3a08fa5f-090f-4d7c-b34a-8823d975f46f'...,"[{'count': 1, 'name': 'alternative'}]",NaN
2,6956c27e-3c15-494f-bff9-af2241f2506c,100,Knocked Up,430346.0,NaN,None,"[{'name': 'Kings of Leon', 'artist': {'id': '6...",2007-03-30,[{'id': 'd0f70ea1-efd5-45e4-8199-6007b351277e'...,"[{'count': 4, 'name': 'rock'}, {'count': 1, 'n...",[USRC10700006]
3,a5412c8d-014c-4379-a517-f6787a85f2d3,100,That Was Just Your Life,430000.0,"live, 2009-04-02: Palais Omnisports de Paris-B...",None,"[{'name': 'Metallica', 'artist': {'id': '65f4f...",2009-04-02,[{'id': 'cee44d1e-e3f9-4e9c-9743-770200c1080a'...,NaN,NaN
4,2119f5c9-1954-4aed-9636-e3c2f65214c3,100,There’ll Never Be Another You,217960.0,NaN,None,"[{'name': 'Lester Young', 'artist': {'id': 'd6...",1959,[{'id': '6b42a2ef-e558-4831-918c-b229abe10521'...,"[{'count': 1, 'name': 'cool jazz'}, {'count': ...",[USGR19906511]
...,...,...,...,...,...,...,...,...,...,...,...
3473,d39d2180-2654-4415-bdc0-ee4373b164c4,100,"Relax, Take It Easy",338613.0,"live, 2009: Sadler's Wells Theatre, London, UK",None,"[{'name': 'MIKA', 'artist': {'id': '8a9ac1cb-f...",2009-09-18,[{'id': '4dfb022c-4216-4f33-b6ec-c83c8dd1f2f5'...,"[{'count': 3, 'name': 'pop'}, {'count': 1, 'na...",[USC7R0900197]
3474,f33fc4cb-0b10-4666-9c12-20ec39ea71d7,100,Lazy Eye,260000.0,NaN,None,"[{'name': 'Silversun Pickups', 'artist': {'id'...",2007-03-06,[{'id': '14c025ab-508a-41f0-98ea-641b27678b52'...,"[{'count': 1, 'name': 'rock'}, {'count': 1, 'n...",NaN
3475,e53b9c87-fa13-4b9d-b727-85e1b8d01eea,97,Monks,201000.0,NaN,None,"[{'name': 'Frank Ocean', 'artist': {'id': 'e52...",2012-07-10,[{'id': 'a942130d-bf15-4a5b-9903-a2bd69792e0a'...,"[{'count': 3, 'name': 'pop'}, {'count': 1, 'na...",[USUM71207188]
3476,da22fa3b-ee21-4326-adda-cfb2dabaaecf,97,This Fire,412000.0,"live, 2014-03-14: Roundhouse, London, UK",None,"[{'name': 'Franz Ferdinand', 'artist': {'id': ...",2014-03-14,[{'id': '88009ca8-9929-4cb8-8614-9f22531d4f6b'...,"[{'count': 2, 'name': 'rock'}, {'count': 2, 'n...",[GBC9X1100683]


In [84]:
set(mb_df['id'])

{'15eb9506-dc2a-427c-bbc3-9eac60e47991',
 '2119f5c9-1954-4aed-9636-e3c2f65214c3',
 '2bfbc6ff-71c5-4af0-a08c-f005e2b725bc',
 '2d6f51c6-afc5-457e-9c58-689679c08d30',
 '310fa155-f188-46bb-b39a-f5ab8ac048df',
 '3245d20e-b10a-4316-8ef2-ebd1b918a25a',
 '34c39c57-0e3a-4a9d-84e7-2f96ba9b665f',
 '40a43f5e-d4f4-4253-a42d-364d47208c64',
 '4aebfdea-4cca-4ff9-ad67-346e8e81944a',
 '5838f978-0822-4e28-874f-e1511324ec3a',
 '6956c27e-3c15-494f-bff9-af2241f2506c',
 '82014f36-3cc7-4682-bfea-2f6b7104c61d',
 '8e25b4da-cd53-45d8-9c86-aa0d72d8b5ce',
 '9b97d9ad-ca8c-445d-ab88-fef4c2b27d6d',
 '9feb4868-d02e-48a9-b24f-dc0cd7a7f7e9',
 'a1f79f80-7190-4881-ad82-724a351da3d9',
 'a5412c8d-014c-4379-a517-f6787a85f2d3',
 'aa12dc94-77b1-4e9b-990f-b2bae959f5e0',
 'e528371a-0544-4328-a51e-8a2427671fcc'}

In [86]:
set(raw_ab_data.keys())

{'15eb9506-dc2a-427c-bbc3-9eac60e47991',
 '2119f5c9-1954-4aed-9636-e3c2f65214c3',
 '2bfbc6ff-71c5-4af0-a08c-f005e2b725bc',
 '2d6f51c6-afc5-457e-9c58-689679c08d30',
 '2df08900-e255-4a99-9f3b-ed10c338b078',
 '310fa155-f188-46bb-b39a-f5ab8ac048df',
 '3245d20e-b10a-4316-8ef2-ebd1b918a25a',
 '34c39c57-0e3a-4a9d-84e7-2f96ba9b665f',
 '40a43f5e-d4f4-4253-a42d-364d47208c64',
 '4aebfdea-4cca-4ff9-ad67-346e8e81944a',
 '5838f978-0822-4e28-874f-e1511324ec3a',
 '6956c27e-3c15-494f-bff9-af2241f2506c',
 '7ae25c17-795d-42d8-8024-07316dd1b87a',
 '7ddbf79a-a65d-4f72-99c0-5c7cd5eed298',
 '82014f36-3cc7-4682-bfea-2f6b7104c61d',
 '8bfd6ab4-0110-4d08-8798-fc60b30f959e',
 '8e25b4da-cd53-45d8-9c86-aa0d72d8b5ce',
 '9b97d9ad-ca8c-445d-ab88-fef4c2b27d6d',
 '9feb4868-d02e-48a9-b24f-dc0cd7a7f7e9',
 'a1159ea1-68bb-4478-91cd-114a5ff1d859',
 'a1f79f80-7190-4881-ad82-724a351da3d9',
 'a5412c8d-014c-4379-a517-f6787a85f2d3',
 'aa12dc94-77b1-4e9b-990f-b2bae959f5e0',
 'ceb7851b-edf1-47a1-94e3-adb8f09d1685',
 'e528371a-0544-

In [105]:
raw_ab_data['2119f5c9-1954-4aed-9636-e3c2f65214c3']['0']['metadata']

{'audio_properties': {'analysis_sample_rate': 44100,
  'bit_rate': 0,
  'codec': 'flac',
  'downmix': 'mix',
  'equal_loudness': 0,
  'length': 217.440002441,
  'lossless': 1,
  'md5_encoded': '9f53f2c7c82c4c001e46106620f65c8c',
  'replay_gain': -8.45418548584},
 'tags': {'acoustid_id': ['73a4c17a-d642-491b-a7c1-93d06c8d2055'],
  'album': ['Jazz in Paris: Le dernier message de Lester Young'],
  'albumartist': ['Lester Young'],
  'albumartistsort': ['Young, Lester'],
  'artist': ['Lester Young'],
  'artistsort': ['Young, Lester'],
  'asin': ['B00005UCHP'],
  'barcode': ['731458955726'],
  'catalognumber': ['589 557-2'],
  'date': ['2002'],
  'discnumber': ['1'],
  'disctotal': ['1'],
  'file_name': '11 There Will Never Be Another You.flac',
  'genre': ['Jazz-In-Paris'],
  'isrc': ['USGR19906511'],
  'label': ['Gitanes Jazz Productions'],
  'media': ['CD'],
  'musicbrainz_albumartistid': ['d613e4ae-093a-49a1-b06a-579480f7f7e8'],
  'musicbrainz_albumid': ['af096bc7-116c-4986-8081-9caa4dd4

In [89]:
mb_data[0]

{'id': 'a1f79f80-7190-4881-ad82-724a351da3d9',
 'score': 100,
 'title': 'Golden Skans',
 'length': 197000,
 'disambiguation': 'live, The Black Sessions',
 'video': None,
 'artist-credit': [{'name': 'Klaxons',
   'artist': {'id': '63011a8d-0117-4f7e-9991-1ef1f337ff70',
    'name': 'Klaxons',
    'sort-name': 'Klaxons',
    'aliases': [{'sort-name': 'The Klaxons',
      'name': 'The Klaxons',
      'locale': None,
      'type': None,
      'primary': None,
      'begin-date': None,
      'end-date': None}]}}],
 'first-release-date': '2006-11-12',
 'releases': [{'id': '6c91736e-8179-41e8-8a31-99b4e28840f7',
   'status-id': '1156806e-d06a-38bd-83f0-cf2284a808b9',
   'count': 1,
   'title': 'The Black Sessions',
   'status': 'Bootleg',
   'artist-credit': [{'name': 'Klaxons',
     'artist': {'id': '63011a8d-0117-4f7e-9991-1ef1f337ff70',
      'name': 'Klaxons',
      'sort-name': 'Klaxons'}}],
   'release-group': {'id': 'ac1d33f3-0035-4c4b-9ef0-df0eae6028aa',
    'type-id': '6fd474e2-6b58-3